In [1]:
# sql_queries.py
import mysql.connector
from datetime import datetime, timedelta

In [3]:
# Database Configuration
DB_CONFIG = {
    'host': '127.0.0.1',  # Or the correct host if your MySQL server is remote
    'user': 'root',        # Replace with your MySQL username
    'password': '12345678', # Replace with your MySQL password
    'database': 'zomato_db',  # The database you created
}

In [4]:
def connect_to_db():
    try:
        mydb = mysql.connector.connect(**DB_CONFIG)
        mycursor = mydb.cursor(dictionary=True)  # Use dictionary=True for easier data access
        return mydb, mycursor
    except mysql.connector.Error as err:
        print(f"Error connecting to database: {err}")
        return None, None

In [5]:
# 1. Peak Ordering Times
def get_peak_ordering_times():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                HOUR(order_date) AS order_hour,
                COUNT(*) AS order_count
            FROM
                orders
            GROUP BY
                order_hour
            ORDER BY
                order_count DESC
            LIMIT 5;  -- Top 5 peak hours
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()


In [6]:
# 2. Locations with Highest Order Volume
def get_top_order_locations():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                c.location,
                COUNT(o.order_id) AS order_count
            FROM
                orders o
            JOIN
                customers c ON o.customer_id = c.customer_id
            GROUP BY
                c.location
            ORDER BY
                order_count DESC
            LIMIT 5;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [7]:
# 3. Restaurants with the Most Orders
def get_top_restaurants():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                r.name,
                COUNT(o.order_id) AS order_count
            FROM
                orders o
            JOIN
                restaurants r ON o.restaurant_id = r.restaurant_id
            GROUP BY
                r.name
            ORDER BY
                order_count DESC
            LIMIT 5;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [9]:
# 4. Top Customers by Order Frequency
def get_top_customers_by_frequency():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                c.name,
                c.email,
                COUNT(o.order_id) AS order_count
            FROM
                orders o
            JOIN
                customers c ON o.customer_id = c.customer_id
            GROUP BY
                c.customer_id
            ORDER BY
                order_count DESC
            LIMIT 5;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [10]:

# 5. Top Customers by Total Spending
def get_top_customers_by_spending():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                c.name,
                c.email,
                SUM(o.total_amount) AS total_spent
            FROM
                orders o
            JOIN
                customers c ON o.customer_id = c.customer_id
            GROUP BY
                c.customer_id
            ORDER BY
                total_spent DESC
            LIMIT 5;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [11]:
# 6. Average Order Value by Cuisine Type
def get_average_order_value_by_cuisine():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                r.cuisine_type,
                AVG(o.total_amount) AS average_order_value
            FROM
                orders o
            JOIN
                restaurants r ON o.restaurant_id = r.restaurant_id
            GROUP BY
                r.cuisine_type
            ORDER BY
                average_order_value DESC;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [12]:
# 7.  Delivery Time Analysis (Average and Percentage of Delayed Deliveries)
def delivery_time_analysis():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                AVG(TIME_TO_SEC(TIMEDIFF(delivery_time, order_date))) / 60 AS average_delivery_time_minutes,
                SUM(CASE WHEN TIMEDIFF(delivery_time, order_date) > SEC_TO_TIME(estimated_time * 60) THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS delayed_delivery_percentage
            FROM
                orders o
            JOIN
                deliveries d ON o.order_id = d.order_id
            WHERE
                o.status = 'Delivered';
        """)
        results = mycursor.fetchone()  # Expecting only one row
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [13]:
# 8.  Average Rating by Restaurant
def get_average_rating_by_restaurant():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                r.name,
                AVG(o.feedback_rating) AS average_rating
            FROM
                orders o
            JOIN
                restaurants r ON o.restaurant_id = r.restaurant_id
            WHERE
                o.feedback_rating IS NOT NULL  -- Exclude orders without ratings
            GROUP BY
                r.name
            ORDER BY
                average_rating DESC
            LIMIT 10; -- Top 10 rated restaurants
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()


In [14]:
# 9.  Delivery Person Performance (Average Delivery Time and Number of Deliveries)
def delivery_person_performance():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                d.delivery_person_id,
                AVG(d.delivery_time) AS average_delivery_time_minutes,
                COUNT(*) AS total_deliveries
            FROM
                deliveries d
            GROUP BY
                d.delivery_person_id
            ORDER BY
                average_delivery_time_minutes ASC  -- Assuming lower is better
            LIMIT 10;  -- Top 10 delivery persons
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [15]:
# 10. Orders by Payment Method
def get_orders_by_payment_method():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None
    try:
        mycursor.execute("""
            SELECT
                payment_mode,
                COUNT(*) AS order_count
            FROM
                orders
            GROUP BY
                payment_mode
            ORDER BY
                order_count DESC;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [16]:
# 11.  Canceled Orders Analysis
def canceled_orders_analysis():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                DATE(order_date) AS order_date,
                COUNT(*) AS canceled_order_count
            FROM
                orders
            WHERE
                status = 'Cancelled'
            GROUP BY
                order_date
            ORDER BY
                order_date DESC;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [17]:
# 12. Discount Analysis
def discount_analysis():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                AVG(discount_applied) AS average_discount,
                SUM(discount_applied) AS total_discount_given
            FROM
                orders;
        """)
        results = mycursor.fetchone()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [18]:

# 13. Average Total Amount by Restaurant
def get_average_total_amount_by_restaurant():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                r.name,
                AVG(o.total_amount) AS average_total_amount
            FROM
                orders o
            JOIN
                restaurants r ON o.restaurant_id = r.restaurant_id
            GROUP BY
                r.name
            ORDER BY
                average_total_amount DESC
            LIMIT 10;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [19]:
# 14.  Customer Order Patterns - Orders per Day of the Week
def customer_order_patterns():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                DAYNAME(order_date) AS day_of_week,
                COUNT(*) AS order_count
            FROM
                orders
            GROUP BY
                day_of_week
            ORDER BY
                order_count DESC;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [20]:
# 15.  Restaurant Order Frequency by Cuisine Type
def restaurant_order_frequency_by_cuisine():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                r.cuisine_type,
                COUNT(o.order_id) AS order_count
            FROM
                orders o
            JOIN
                restaurants r ON o.restaurant_id = r.restaurant_id
            GROUP BY
                r.cuisine_type
            ORDER BY
                order_count DESC;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [21]:
# 16.  Delivery Time vs. Distance
def delivery_time_vs_distance():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                d.distance,
                AVG(d.delivery_time) AS avg_delivery_time
            FROM
                deliveries d
            GROUP BY
                d.distance
            ORDER BY
                d.distance;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()


In [22]:
# 17. Premium Customer Analysis
def premium_customer_analysis():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                c.is_premium,
                COUNT(o.order_id) AS order_count,
                AVG(o.total_amount) AS average_order_value
            FROM
                orders o
            JOIN
                customers c ON o.customer_id = c.customer_id
            GROUP BY
                c.is_premium;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [23]:
# 18. Average Orders per Customer
def average_orders_per_customer():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                AVG(customer_orders) AS average_orders
            FROM (
                SELECT
                    customer_id,
                    COUNT(*) AS customer_orders
                FROM
                    orders
                GROUP BY
                    customer_id
            ) AS customer_order_counts;
        """)
        results = mycursor.fetchone()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [24]:
# 19. Percentage of Orders by Status
def orders_by_status():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                status,
                COUNT(*) AS order_count,
                (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM orders)) AS percentage
            FROM
                orders
            GROUP BY
                status;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()

In [25]:
# 20. Top Cuisines by Number of Orders
def top_cuisines_by_orders():
    mydb, mycursor = connect_to_db()
    if not mycursor:
        return None

    try:
        mycursor.execute("""
            SELECT
                r.cuisine_type,
                COUNT(o.order_id) AS order_count
            FROM
                orders o
            JOIN
                restaurants r ON o.restaurant_id = r.restaurant_id
            GROUP BY
                r.cuisine_type
            ORDER BY
                order_count DESC
            LIMIT 5;
        """)
        results = mycursor.fetchall()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if mydb:
            mydb.close()